In [2]:
import numpy as np
import json

datafile = r"Z:\shot_data\19-615.txt"

with open(datafile) as f:
    data = json.load(f)

#locals function unpacks dictionary keys into local variables.
print(data)
locals().update(data)

#copper Us-up fit values, do not change
C_Cu = 3.970
S_Cu = 1.479


{'vproj': 1.979, 'vproj_err': 0.006, 'pin1': [-2.81, -9.472, 0], 'pin2': [-6.204, 6.69, 0], 'pin3': [9.486, 1.91, 0], 'pin4': [0, 0, 0], 't1': 18.321, 't2': 18.32724, 't3': 18.33564, 't4': 18.61148, 'rho_0': 2.708, 'rho_0_err': 0.001, 'rho_0_imp': 8.944, 'rho_0_imp_err': 0.003, 'U_s': 7.2488, 'U_s_err': 0.0007}


In [3]:
#doing impedance matching correctly this time...
#For each shot, use rho_0 and U_s to be slope of P-up line for TPX, then use determined copper P-up line
#find intersection, that is P and up value

def intersect(rho_0, U_s, rho_0_imp, C_Cu, S_Cu, vproj):
    slope = rho_0 * U_s
    a = rho_0_imp*S_Cu
    b = -rho_0_imp*C_Cu -2*rho_0_imp*S_Cu*vproj
    c = rho_0_imp*C_Cu*vproj + rho_0_imp*S_Cu*vproj**2

    #Find intersection of line and parabola - use substitution and then quadratic equation to solve
    d = b-slope
    x1 = (-d + np.sqrt(d**2 - 4*a*c))/(2*a)
    x2 = (-d - np.sqrt(d**2 - 4*a*c))/(2*a)

    # choose correct (realistic) u_p value and plug into tpx equation to get P

    P1 = slope*x1
    P2 = slope*x2
    return x1, P1, x2, P2

In [4]:
#l = [x1, P1, x2, P2]
#[i if i>0 else None for i in l]

def positive(x, P):
    if x >= 0 and P >= 0:
        return True
    else:
        return False
   

In [5]:
x1, P1, x2, P2 = intersect(rho_0, U_s, rho_0_imp, C_Cu, S_Cu, vproj)
if positive(x2, P2)==True and positive(x1, P1)==True:
    print("Two intersections:\n up = " + str(x1) + " and P = " + str(P1) + "\n up = " + str(x2) + " and P = " + str(P2))
elif positive(x1, P1)==True:
    print("up = " + str(x1) + " and P = " + str(P1))
elif positive(x2, P2)==True:
    print("up = " + str(x2) + " and P = " + str(P2))
elif positive(x2, P2)==False and positive(x1, P1)==False:
    print("no intersection")
else:
    print("error")

Two intersections:
 up = 6.761263883015255 and P = 132.7219224121243
 up = 1.3649170060203935 and P = 26.792980144895623


In [15]:
#choose which of the intersections above is reasonable (if more than one exists)
up = x2
P = P2

#calculate density in shocked state
shocked_density = (rho_0*U_s) / (U_s-up)
print("Impedance matching results: \n up = " + str(format(up, '.4f')) + " (km/s) \n P = "+str(format(P, '.4f'))+" (GPa) \n rho = "+str(format(shocked_density, '.4f'))+" (g/cc)")

Impedance matching results: 
 up = 1.3649 (km/s) 
 P = 26.7930 (GPa) 
 rho = 3.3362 (g/cc)


In [14]:
#Monte carlo to get errors
import itertools

#from Knudson alpha quartz paper, do not change
C_err = np.sqrt(2.9250*10**-4)
S_err = np.sqrt(5.0020*10**-4)
CS_err = 3.7920*10**-4 #correlation
r = CS_err/(C_err*S_err) #correlation coefficient



def monte(U_s, U_s_err, vproj, vproj_err):
    for _ in itertools.repeat(None, 100000):
        z1 = np.random.normal(0, 1, 1) #mean 0, stdev 1, 1 value
        z2 = np.random.normal(0, 1, 1)

        C_ = np.random.normal(C_Cu, C_err)
        S_ = np.random.normal(S_Cu, S_err)

        C_1 = C_ + C_err*(r*z2 + (z1*np.sqrt(1-r**2)))
        S_1 = S_ + S_err*z2
        #now C_1 and S_1 take correlation and variability into account. Use these in the rest of the calculation.

        rhozero = np.random.normal(rho_0, rho_0_err)
        shockspeed = np.random.normal(U_s, U_s_err)
        rhozeroimp = np.random.normal(rho_0_imp, rho_0_imp_err)
        projvelocity = np.random.normal(vproj, vproj_err)
        
        #solve equations with randomized values
        x1, P1, x2, P2 = intersect(rhozero, shockspeed, rhozeroimp, C_1, S_1, projvelocity)
        
    
        Ps.append(P2)
        ups.append(x2)
        
        rho_1_ = (rhozero*shockspeed)/(shockspeed - x2)
        rhos.append(rho_1_)

        


Ps = []
ups = []
rhos = []

monte(U_s, U_s_err, vproj, vproj_err)
print("Mean P: " + str(format(np.mean(Ps), '.4f')) + "\nP Standard deviation: " + str(format(np.std(Ps), '.4f')))
print("Mean up: " + str(format(np.mean(ups), '.4f')) + "\nup Standard deviation: " + str(format(np.std(ups), '.4f')))
print("Mean rho_1: " + str(format(np.mean(rhos), '.4f')) + "\nRho Standard deviation: " + str(format(np.std(rhos), '.4f')) + "\n")

Mean P: 26.7927
P Standard deviation: 0.1032
Mean up: 1.3649
up Standard deviation: 0.0052
Mean rho_1: 3.3362
Rho Standard deviation: 0.0032

